In [1]:
from mypulp import * 
import itertools

# 容量制約付き施設配置問題

In [10]:
def flp(I, J, d, M, f, c):
    model = Model('flp')
    
    ## 変数 ##
    x, y = {}, {}
    for j in J:
        y[j] = model.addVar(vtype='B', name=f'y_{j}')
        for i in I:
            x[i, j] = model.addVar(vtype='C', name=f'x_{i},{j}')
    model.update()
    
    ## 定式化 ##
    for i in I:
        model.addConstr(quicksum(x[i, j] for j in J) == d[i])
    for j in J:
        model.addConstr(quicksum(x[i, j] for i in I) <= M[j] * y[j])
    for i, j in itertools.product(I, J):
        model.addConstr(x[i, j] <= d[i] * y[j])
    model.setObjective(quicksum(f[j] * y[j] for j in J) 
                       + quicksum(c[i][j] * x[i, j] for j in J for i in I), GRB.MINIMIZE)
    model.update()
    
    return model    

In [11]:
## 定数 ##
# 顧客iの需要量
d = [80, 270, 250, 160, 180]
# 施設jの開設費用
f = [1000, 1000, 1000]
# 施設jの容量
M = [500, 500, 500]
# 施設jから顧客jに1単位輸送するときにかかるコスト
c = [[4, 6, 9], [5, 4, 7], [6, 3, 4],
     [8, 5, 3], [10, 8, 4]]
I = range(len(d))
J = range(len(f))

## 最適化実行 ##
model = flp(I, J, d, M, f, c)
model.optimize()
print('Optimal value: ', model.ObjVal)
for v in model.getVars():
    print(f'{v.VarName}: {v.X}')

Optimal value:  5610.0
x_0,0: 0.0
x_0,1: 80.0
x_0,2: 0.0
x_1,0: 0.0
x_1,1: 270.0
x_1,2: 0.0
x_2,0: 0.0
x_2,1: 150.0
x_2,2: 100.0
x_3,0: 0.0
x_3,1: 0.0
x_3,2: 160.0
x_4,0: 0.0
x_4,1: 0.0
x_4,2: 180.0
y_0: 0.0
y_1: 1.0
y_2: 1.0


# p-Median

In [41]:
# 人口，距離データ読み込み
w = np.loadtxt("demand_nara2013.csv")
d = np.loadtxt("dismat_daihyo.csv", delimiter=",")

I = range(len(demand))
J = range(len(demand))

# 配置施設数
p = 4

In [42]:
def solve_pMedian(I, J, w, d, p):
    model = Model('pMedian')
    
    ## 変数 ##
    x, y = {}, {}
    for j in J:
        y[j] = model.addVar(vtype='B', name=f'y_{j}')
        for i in I:
            x[i, j] = model.addVar(vtype='B', name=f'x_{i},{j}')
            
    model.update()
    
    ## 定式化 ##
    for i in I:
        model.addConstr(quicksum(x[i, j] for j in J) == 1)
    
    model.addConstr(quicksum(y[j] for j in J) == p)
    
    for i, j in itertools.product(I, J):
        model.addConstr(x[i, j] <= y[j])
    
    model.setObjective(quicksum(w[i] * d[i][j] * x[i, j] for j in J for i in I), GRB.MINIMIZE)
    
    model.update()
    model.__data = x, y

    return model    

In [43]:
model = solve_pMedian(I, J, w, d, p)
model.optimize()
x, y = model.__data

# 最適解
print(f'Optimal Value: {model.ObjVal/1000: .3f} km')
print('Facilities are located at:')
eps = 1.0e-6
for j in y:
    if y[j].X > eps:
        print(y[j])

Optimal Value:  5628359.795 km
Facilities are located at:
y_0
y_4
y_8
y_24


# p-Center

In [45]:
# 距離データ読み込み
d = np.loadtxt("dismat_daihyo.csv", delimiter=",")

I = range(len(demand))
J = range(len(demand))

# 配置施設数
p = 4

In [50]:
def solve_pCenter(I, J, d, p):
    model = Model('pCenter')
    
    ## 変数 ##
    x, y = {}, {}
    for j in J:
        y[j] = model.addVar(vtype='B', name=f'y_{j}')
        for i in I:
            x[i, j] = model.addVar(vtype='B', name=f'x_{i},{j}')
    z = model.addVar(vtype='C', name='z')
            
    model.update()
    
    ## 定式化 ##
    for i in I:
        model.addConstr(quicksum(x[i, j] for j in J) == 1)
        model.addConstr(quicksum(d[i][j] * x[i, j] for j in J) <= z)
    model.addConstr(quicksum(y[j] for j in J) == p)
    
    for i, j in itertools.product(I, J):
        model.addConstr(x[i, j] <= y[j])
    
    model.setObjective(z, GRB.MINIMIZE)
    
    model.update()
    model.__data = x, y, z

    return model    

In [51]:
model = solve_pCenter(I, J, d, p)
model.optimize()
x, y, z = model.__data

# 最適解
print(f'Optimal Value: {model.ObjVal/1000: .3f} km')
print('Facilities are located at:')
eps = 1.0e-6
for j in y:
    if y[j].X > eps:
        print(y[j])

Optimal Value:  21.332 km
Facilities are located at:
y_6
y_11
y_15
y_35


最大値を最小化するタイプの定式化は避けたい -> 2分探索を利用した定式化

In [110]:
def solve_pCover(I, J, a, d, p):
    model = Model('pCover')
    
    ## 変数 ##
    y, z = {}, {}
    for i in I:
        z[i] = model.addVar(vtype='B', name=f'z_{i}')
    for j in J:
        y[j] = model.addVar(vtype='B', name=f'y_{j}')
            
    model.update()
    
    ## 定式化 ##
    for i in I:
        model.addConstr(quicksum(a[i][j] * y[j] for j in J) + z[i] >= 1)

    model.addConstr(quicksum(y[j] for j in J) == p)
        
    model.setObjective(quicksum(z[i] for i in I), GRB.MINIMIZE)
    
    model.update()
    model.__data = y, z

    return model    

In [111]:
def solve_pCenter2(I, J, d, p):
    # 初期値
    UB = d.max()
    LB = 0
    
    while UB - LB > 1.0e-4:
        theta = (UB + LB) / 2
        a = [[1 if d[i][j] < theta else 0 for i in I] for j in J]
        
        model = solve_pCover(I, J, a, d, p)
        model.optimize()
        y, z = model.__data
        infeasibility = sum([z[i].X for i in I])
        
        if infeasibility > 0:  # このthetaとpではカバーできない -> 最適解はtheta以上
            LB = theta
        else:
            UB = theta
        
    model.__data = y, z
    
    return model, theta  

In [112]:
model, theta = solve_pCenter2(I, J, d, p)
y, z = model.__data

# 最適解
print(f'Optimal Value: {theta/1000: .3f} km')
print('Facilities are located at:')
eps = 1.0e-6
for j in y:
    if y[j].X > eps:
        print(y[j])

Optimal Value:  21.332 km
Facilities are located at:
y_3
y_33
y_35
y_38
